In [24]:
import sys
print(sys.executable)

/Users/cbarril/dev/posgrado/vpc2_19co2025/vpc2/bin/python


In [25]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import pandas as pd

import torch
import torchvision
import torchsummary
import torchmetrics
import cv2 as cv

from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split

from src.maps import species_es_map, disease_es_map

In [26]:
# Definir carpeta 'color'
base_dir = '../data/plantvillage/plantvillage dataset'
sub = 'color'
sub_path = os.path.join(base_dir, sub)

data = []

if os.path.exists(sub_path):
    for folder in os.listdir(sub_path):
        folder_path = os.path.join(sub_path, folder)
        if not os.path.isdir(folder_path):
            continue
        species, disease = folder.split('___', 1)
        healthy = disease == 'healthy'
        disease = None if healthy else disease
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                data.append({
                    'Format': sub,
                    'Species': species,
                    'Healthy': healthy,
                    'Disease': disease,
                    'FileName': file
                })

# Crear DataFrame
df = pd.DataFrame(data)

# Agregar nombre en español para especie
df['Especie'] = df['Species'].map(species_es_map)

# Agregar nombre en español para enfermedad (o 'Sano' si es healthy)
df['Enfermedad'] = df.apply(
    lambda row: 'Sano' if row['Healthy'] else disease_es_map.get(row['Disease'], row['Disease']),
    axis=1
)

df.head()

,Format,Species,Healthy,Disease,FileName,Especie,Enfermedad
0,color,Strawberry,True,None,8f558908-aa1b-4a86-855a-5094c2392e5a___RS_HL 1...,Fresa,Sano
1,color,Strawberry,True,None,b8e9ed27-8e37-4214-9206-f8c0ef21cf4d___RS_HL 4...,Fresa,Sano
2,color,Strawberry,True,None,abdd34a0-ab02-41e0-95a3-a014ab863ec2___RS_HL 1...,Fresa,Sano
3,color,Strawberry,True,None,d1aee44a-b6bb-45b9-b7b6-5d553add8fd1___RS_HL 2...,Fresa,Sano
4,color,Strawberry,True,None,3d28c3ea-8419-4e09-addd-211e3828e39f___RS_HL 1...,Fresa,Sano


In [27]:
import cv2 as cv
import random

# Sample a few images from the DataFrame
sample_images = df.sample(5, random_state=42)

for _, row in sample_images.iterrows():
    folder = f"{row['Species']}___{'healthy' if row['Healthy'] else row['Disease']}"
    image_path = os.path.join(base_dir, 'color', folder, row['FileName'])

    img = cv.imread(image_path)
    height, width = img.shape[:2]
    print(f"Image: {row['FileName']}")
    print(f"Original dimensions: {width}x{height}")
    print("-" * 40)

Image: fe9abcc5-51b6-4524-bc31-e56e1683df72___RS_HL 7007.JPG
Original dimensions: 256x256
----------------------------------------
Image: e7b889f3-d580-426e-83fb-6519f43935ef___Com.G_SpM_FL 8690.JPG
Original dimensions: 256x256
----------------------------------------
Image: 7cf98788-6726-455c-9b6b-0163e80d4c0d___GCREC_Bact.Sp 3733.JPG
Original dimensions: 256x256
----------------------------------------
Image: bb649ab8-c07f-4d62-a186-e2cc4963dd6b___Matt.S_CG 2652.JPG
Original dimensions: 256x256
----------------------------------------
Image: d6cfcd4a-00f3-475b-9e65-0df8b7afbe25___FAM_B.Rot 3132.JPG
Original dimensions: 256x256
----------------------------------------


In [5]:
df

,Format,Species,Healthy,Disease,FileName,Especie,Enfermedad
0,color,Strawberry,True,None,8f558908-aa1b-4a86-855a-5094c2392e5a___RS_HL 1...,Fresa,Sano
1,color,Strawberry,True,None,b8e9ed27-8e37-4214-9206-f8c0ef21cf4d___RS_HL 4...,Fresa,Sano
2,color,Strawberry,True,None,abdd34a0-ab02-41e0-95a3-a014ab863ec2___RS_HL 1...,Fresa,Sano
3,color,Strawberry,True,None,d1aee44a-b6bb-45b9-b7b6-5d553add8fd1___RS_HL 2...,Fresa,Sano
4,color,Strawberry,True,None,3d28c3ea-8419-4e09-addd-211e3828e39f___RS_HL 1...,Fresa,Sano
...,...,...,...,...,...,...,...
54300,color,Soybean,True,None,57c18b39-2a33-471f-91eb-a9ba4ddabc7b___RS_HL 6...,Soja,Sano
54301,color,Soybean,True,None,4fdc663e-a8ea-4d8a-801b-ef18ad192661___RS_HL 6...,Soja,Sano
54302,color,Soybean,True,None,df807f13-078b-4a6a-9c23-e43e540ecdc2___RS_HL 5...,Soja,Sano
54303,color,Soybean,True,None,60bf9858-951a-4b56-906e-3c1b336973ba___RS_HL 4...,Soja,Sano


Especie + Enfermedad como etiqueta

In [ ]:
# df['Label'] = df['Especie'] + ' - ' + df['Enfermedad']
# df['Label_id'] = df['Label'].astype('category').cat.codes
# label_map = dict(enumerate(df['Label'].astype('category').cat.categories))
# NUM_CLASSES = len(label_map)

Solo Enfermedad como etiqueta

In [6]:
df['Label'] = df['Enfermedad']
df['Label_id'] = df['Label'].astype('category').cat.codes
label_map = dict(enumerate(df['Label'].astype('category').cat.categories))
NUM_CLASSES = len(label_map)

In [7]:
# Semilla reproducible
SEED = 42

# Split 70% train, 20% valid, 10% test
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['Label_id'], random_state=SEED)
valid_df, test_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_id'], random_state=SEED)

In [18]:
class PlantVillageDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, format_type='color', transform=None):
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.format_type = format_type
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        folder = f"{row['Species']}___{'healthy' if row['Healthy'] else row['Disease']}"
        image_path = os.path.join(self.root_dir, folder, row['FileName'])

        image = cv.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {image_path}")

        # Convert BGR (OpenCV default) to RGB
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        # Convert to PIL-like tensor manually if transforms require it
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0  # From HWC to CHW and normalize

        if self.transform:
            image = self.transform(image)

        label = self.df.iloc[idx]['Label_id']  # Esto puede ser int o np.int64
        label = torch.tensor(label, dtype=torch.long)  # <-- Convierte explícitamente
        return image, label

In [19]:
import time
from tqdm import tqdm  # para barra de progreso (opcional)

def train(model, optimizer, criterion, metric, data, epochs, tb_writer=None, log_interval=10):
    train_loader = data["train"]
    valid_loader = data["valid"]

    train_writer = tb_writer["train"] if tb_writer else None
    valid_writer = tb_writer["valid"] if tb_writer else None

    if tb_writer:
        dummy_input = torch.zeros((1, 3, data["image_width"], data["image_height"]))
        train_writer.add_graph(model, dummy_input)
        valid_writer.add_graph(model, dummy_input)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    metric.to(device)

    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []

    print(f"Training started on device: {device}")

    for epoch in range(epochs):
        start_epoch_time = time.time()
        model.train()

        epoch_train_loss = 0.0
        epoch_train_accuracy = 0.0

        # Usamos tqdm para barra de progreso en batches
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs} [Training]")

        for batch_idx, (train_data, train_target) in progress_bar:
            train_data = train_data.to(device)
            train_target = train_target.to(device)

            optimizer.zero_grad()
            output = model(train_data.float())
            loss = criterion(output, train_target)
            loss.backward()
            optimizer.step()

            acc = metric(output, train_target)

            epoch_train_loss += loss.item()
            epoch_train_accuracy += acc.item()

            if batch_idx % log_interval == 0:
                avg_loss = epoch_train_loss / (batch_idx + 1)
                avg_acc = epoch_train_accuracy / (batch_idx + 1)
                progress_bar.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{avg_acc:.4f}")

        epoch_train_loss /= len(train_loader)
        epoch_train_accuracy /= len(train_loader)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_accuracy)

        # Validación
        model.eval()
        epoch_valid_loss = 0.0
        epoch_valid_accuracy = 0.0

        with torch.no_grad():
            for valid_data, valid_target in valid_loader:
                valid_data = valid_data.to(device)
                valid_target = valid_target.to(device)

                output = model(valid_data.float())
                loss = criterion(output, valid_target)
                acc = metric(output, valid_target)

                epoch_valid_loss += loss.item()
                epoch_valid_accuracy += acc.item()

        epoch_valid_loss /= len(valid_loader)
        epoch_valid_accuracy /= len(valid_loader)
        valid_loss.append(epoch_valid_loss)
        valid_acc.append(epoch_valid_accuracy)

        epoch_duration = time.time() - start_epoch_time

        print(f"Epoch {epoch+1}/{epochs} completed in {epoch_duration:.1f}s - "
              f"Train Loss: {epoch_train_loss:.6f}, Train Acc: {epoch_train_accuracy:.6f} - "
              f"Valid Loss: {epoch_valid_loss:.6f}, Valid Acc: {epoch_valid_accuracy:.6f}")

        if tb_writer:
            train_writer.add_scalar("loss", epoch_train_loss, epoch)
            valid_writer.add_scalar("loss", epoch_valid_loss, epoch)
            train_writer.add_scalar("accuracy", epoch_train_accuracy, epoch)
            valid_writer.add_scalar("accuracy", epoch_valid_accuracy, epoch)
            train_writer.flush()
            valid_writer.flush()

    print("Training finished.")

    history = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc
    }

    return history

In [20]:
IMAGE_SIZE = (256, 256)

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    # No ToTensor or Normalize needed — already applied
])

val_test_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
])

base_dir = '../data/plantvillage/plantvillage dataset'

train_dataset = PlantVillageDataset(train_df, base_dir, format_type='color', transform=train_transform)
valid_dataset = PlantVillageDataset(valid_df, base_dir, format_type='color', transform=val_test_transform)
test_dataset = PlantVillageDataset(test_df, base_dir, format_type='color', transform=val_test_transform)

BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [21]:
resnet_model = models.resnet18(weights=None)
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, NUM_CLASSES)

In [23]:
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()
metric = torchmetrics.Accuracy(task='multiclass', num_classes=NUM_CLASSES)

data_dict = {
    "train": train_loader,
    "valid": valid_loader,
    "image_width": IMAGE_SIZE[0],
    "image_height": IMAGE_SIZE[1]
}

writer = {
    "train": SummaryWriter(log_dir="runs/plant_train"),
    "valid": SummaryWriter(log_dir="runs/plant_valid")
}

history = train(
    model=resnet_model,
    optimizer=optimizer,
    criterion=criterion,
    metric=metric,
    data=data_dict,
    epochs=30,
    tb_writer=writer
)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

axs[0].plot(history["train_loss"], label='Train Loss')
axs[0].plot(history["valid_loss"], label='Valid Loss')
axs[0].set_title('Loss')
axs[0].legend()

axs[1].plot(history["train_acc"], label='Train Accuracy')
axs[1].plot(history["valid_acc"], label='Valid Accuracy')
axs[1].set_title('Accuracy')
axs[1].legend()

plt.tight_layout()
plt.show()